# Simple linear regression 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class LinearRegression:
    def __init__(self,X,Y): 
        ones=np.ones(X.shape)
        X=np.append(ones,X,axis=1)
        self.X=X
        self.Y=Y
        self.m=X.shape[0]
        self.n=X.shape[1]
        self.theta=np.random.randn(X.shape[1])
        
    def computeCostFunction(self):
        h=np.matmul(self.X,self.theta)
        self.J=(1/(2*self.m))*np.sum((h-self.Y)**2)
        return self.J
    
    def performGradientDescent(self,num_of_iter,alpha):
        self.Cost_history=[]
        self.theta_history=[]
        for x in range(num_of_iter):
            h=np.matmul(self.X,self.theta)
            J=self.computeCostFunction()
            self.Cost_history.append(J)
            self.theta_history.append(self.theta)
            temp=h-self.Y
            self.theta=self.theta-(alpha/self.m)*(self.X.T.dot(temp))
        return self.theta,self.Cost_history,self.theta_history
            
        
    def predict(self,X_test,Y_test):
        ones=np.ones(X_test.shape)
        X_test=np.append(ones,X_test,axis=1)
        self.Y_pred=np.matmul(X_test,self.theta)
        self.error_percentage=(abs(self.Y_pred-Y_test)/Y_test)*100
        return self.Y_pred,self.error_percentage
    
    def predictUsingNormalEquation(self,X_test,Y_test):
        ones=np.ones(X_test.shape)
        X_test=np.append(ones,X_test,axis=1)
        inv=np.linalg.inv(np.matmul(self.X.T,self.X))
        self.w=np.matmul(np.matmul(inv,self.X.T),self.Y)
        y_pred=np.matmul(X_test,self.w)
        return y_pred,(abs(Y_test-y_pred)/Y_test)*100
    
        
    def returnTheta(self):
        return self.theta
    
    def returnX(self):
        return self.X
        
    def returnY(self):
        return self.Y


In [ ]:
class FeatureScaling:
    def __init__(self,X,y):
        self.X=X.copy()
        if y.ndim==1:
            y=np.reshape(y,(y.shape[0],1))
        self.y=y.copy()
        self.minMax_X={}
        self.minMax_y={}
    
    def fit_transform_X(self):
        num_of_features=self.X.shape[1]
        for i in range(num_of_features):
            feature=self.X[:,i]
            Mean=np.mean(feature)
            Min=np.min(feature)
            Max=np.max(feature)
            feature=(feature-Mean)/(Max-Min)
            self.minMax_X[i]=np.array([Mean,Min,Max])
            self.X[:,i]=feature
        return self.X.copy()
    
    def fit_transform_Y(self):
        num_of_features=self.y.shape[1]
        for i in range(num_of_features):
            feature=self.y[:,i]
            Mean=np.mean(feature)
            Min=np.min(feature)
            Max=np.max(feature)
            feature=(feature-Mean)/(Max-Min)
            self.minMax_y[i]=np.array([Mean,Min,Max])
            self.y[:,i]=feature
        return np.reshape(self.y,self.y.shape[0])
    
    def inverse_transform_X(self,X):
        X_transformed=X.copy()
        num_of_features=X_transformed.shape[1]
        for i in range(num_of_features):
            feature=X_transformed[:,i]
            Mean=self.minMax_X[i][0]
            Min=self.minMax_X[i][1]
            Max=self.minMax_X[i][2]
            feature=feature*(Max-Min)+Mean
            X_transformed[:,i]=feature
        return X_transformed
    
    def inverse_transform_Y(self,y):
        y_transformed=y.copy()
        if y_transformed.ndim==1:
            y_transformed=np.reshape(y_transformed,(y_transformed.shape[0],1))
        num_of_features=y_transformed.shape[1]
        for i in range(num_of_features):
            feature=y_transformed[:,i]
            Mean=self.minMax_y[i][0]
            Min=self.minMax_y[i][1]
            Max=self.minMax_y[i][2]
            feature=feature*(Max-Min)+Mean
            y_transformed[:,i]=feature
        return np.reshape(y_transformed,y_transformed.shape[0])
    
    def transform_X(self,X):
        X_transformed=X.copy()
        num_of_features=X_transformed.shape[1]
        for i in range(num_of_features):
            feature=X_transformed[:,i]
            Mean=self.minMax_X[i][0]
            Min=self.minMax_y[i][1]
            Max=self.minMax_y[i][2]
            feature=(feature-Mean)/(Max-Min)
            X_transformed[:,i]=feature
        return X_transformed
    
    def transform_Y(self,y):
        y_transformed=y.copy()
        if y_transformed.ndim==1:
            y_transformed=np.reshape(y_transformed,(y_transformed.shape[0],1))
        num_of_features=y_transformed.shape[1]
        for i in range(num_of_features):
            feature=y_transformed[:,i]
            Mean=self.minMax_y[i][0]
            Min=self.minMax_y[i][1]
            Max=self.minMax_y[i][2]
            feature=(feature-Mean)/(Max-Min)
            y_transformed[:,i]=feature
        return np.reshape(y_transformed,y_transformed.shape[0])
    
    def returnX(self):
        return self.X
    
    def returnY(self):
        return self.y
        
        

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from LinearRegression import LinearRegression

In [ ]:
Data=pd.read_csv('Salary_Data.csv')
print(Data)

In [ ]:
Data.describe()

In [ ]:
#plot of dataset
plt.scatter(Data.iloc[:,0:1].values,Data.iloc[:,1].values)
plt.xlabel('Years of experience')
plt.ylabel('Salary')
plt.title('Plot of Data set')
plt.show()

In [ ]:
#training and testing size
train_size=int(0.7*Data.shape[0])
test_size=int(0.3*Data.shape[0])
print("Training set size : "+ str(train_size))
print("Testing set size : "+str(test_size))

In [ ]:
#shuffle the dataset
Data=Data.sample(frac=1)
X=Data.iloc[:,0:1].values
y=Data.iloc[:,1].values

In [ ]:
from FeatureScaling import FeatureScaling
fs=FeatureScaling(X,y)
X=fs.fit_transform_X()
y=fs.fit_transform_Y()

In [ ]:
#training set split
X_train=X[0:train_size,:]
Y_train=y[0:train_size]

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
#testing set split
X_test=X[train_size:,:]
Y_test=y[train_size:]

In [ ]:
print(X_test.shape)
print(Y_test.shape)

In [ ]:
#scatter plot of training set
plt.scatter(X_train,Y_train)
plt.xlabel('Years of experience')
plt.ylabel('Salary')
plt.title('Plot of training set')
plt.show()

In [ ]:
#scatter plot of testing set
plt.scatter(X_test,Y_test)
plt.xlabel('Years of experience')
plt.ylabel('Salary')
plt.title('Plot of testing set')
plt.show()

In [ ]:
lr=LinearRegression(X_train,Y_train)

In [ ]:
theta=lr.returnTheta()
print(theta)

In [ ]:
#testing set prediction
y_pred_normal,error_percentage=lr.predictUsingNormalEquation(X_test,Y_test)
y_pred_normal=fs.inverse_transform_Y(y_pred_normal)
print(error_percentage)

In [ ]:
#training set prediction
y_pred_train_normal,error_percentage_train_normal=lr.predictUsingNormalEquation(X_train,Y_train)
y_pred_train_normal=fs.inverse_transform_Y(y_pred_train_normal)
print(lr.computeCostFunction())

In [ ]:
#learning parameters
n_iter=1000
alpha=0.05

theta,J_Array,theta_array=lr.performGradientDescent(n_iter,alpha)

In [ ]:
y_pred_grad,ErrorPercentage=lr.predict(X_test,Y_test)
print(ErrorPercentage)
y_pred_grad=fs.inverse_transform_Y(y_pred_grad)

In [ ]:
#let's see how train set is predicted
y_pred_train,error_for_train=lr.predict(X_train,Y_train)
y_pred_train=fs.inverse_transform_Y(y_pred_train)
print(error_for_train)

In [ ]:
#inverse scaling the features
X_train=fs.inverse_transform_X(X_train)
Y_train=fs.inverse_transform_Y(Y_train)
X_test=fs.inverse_transform_X(X_test)
Y_test=fs.inverse_transform_Y(Y_test)

In [ ]:
#let's see how train set is predicted using gradient descent
plt.scatter(X_train,Y_train)
plt.plot(X_train,y_pred_train,'r')
plt.xlabel('Years of experience')
plt.ylabel('Salary')
plt.title('Training set prediction using Gradient Descent')
plt.show()

In [ ]:
#let's see how train set is predicted using normal equation
plt.scatter(X_train,Y_train)
plt.plot(X_train,y_pred_train_normal,'r')
plt.xlabel('X_axis')
plt.ylabel('Y_axis')
plt.title('Training set prediction using Normal Equation')
plt.show()

In [1]:
#let's see how test set is predicted using gradient descent
plt.scatter(X_test,Y_test)
plt.plot(X_test,y_pred_grad,'r')
plt.xlabel('X_axis')
plt.ylabel('Y_axis')
plt.title('Test set prediction using Gradient Descent')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
#let's see how test set is predicted using normal equation method
plt.scatter(X_test,Y_test)
plt.plot(X_test,y_pred_normal,'r')
plt.xlabel('X_axis')
plt.ylabel('Y_axis')
plt.title('Test set prediction using Normal Equation')
plt.show()

In [ ]:
#plot of how cost function is minimized as number of iterations is proceeded
x=[i for i in range(1000)]
plt.plot(x,J_Array)
plt.xlabel('Number of iterations')
plt.ylabel('Cost function(J)')
plt.show()